In [1]:
# !pip install easyocr

In [1]:
import os
import cv2
import sys,glob
import easyocr
import torch

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
import re
import numpy as np
import pandas as pd
import dateutil

In [4]:
from datetime import date
from datetime import datetime
from dateutil.parser import parse

# Sample Prescription text-

In [5]:
prescription_text = "Patient John Doe received a prescription, for the following medication. The medicine name is Acetaminophen, with a dosage of 500mg, Mft 2021-08-03 and This prescription aims to manage discomfort and ensure the patient's well-being Exp 2026-08-03. The price of the medicine is Rs 50.00]	"

Medicine name, Dosage, Price

In [6]:
medicine_name_match = re.search(r'Medicine name is (\w+)', prescription_text, re.IGNORECASE)
if medicine_name_match:
    medicine_name = medicine_name_match.group(1)
else:
    medicine_name = "Not found"

dosage_match = re.search(r'dosage of (\d+(?:mg|g)?)', prescription_text, re.IGNORECASE)
if dosage_match:
    dosage = dosage_match.group(1)
else:
    dosage = "Not found"

price_match = re.search(r'(?:Rs|₹)?\s?(\d+\.\d{2})', prescription_text)
if price_match:
    price = price_match.group(1)
else:
    price = "Not found"

In [7]:
print("Medicine Name:", medicine_name)
print("Dosage/Strength:", dosage)
print("Price:", price)

Medicine Name: Acetaminophen
Dosage/Strength: 500mg
Price: 50.00


Extract Expiry date

In [8]:
def datepart(lst):
    lst = [ls.replace('.', '-') for ls in lst]
    lst = set(lst)
    lst = list(lst)

    today = date.today()
    tdy = str(today)
    exp_date = ""

    for date_str in lst:
        if date_str > tdy and date_str > exp_date:
            exp_date = date_str

    return exp_date

dates = re.findall(r'(\d{4}-\d{2}-\d{2})', prescription_text)
exp_date = datepart(dates)

In [9]:
dates = re.findall(r'(\d{4}-\d{2}-\d{2})', prescription_text)
exp_date = datepart(dates)

In [10]:
print("Expiration Date:", exp_date)

Expiration Date: 2026-08-03


# Sample Images

In [11]:
reader = easyocr.Reader(['en', 'en'])

In [12]:
img_two_text = reader.readtext('/content/img_two.jpg')
final_text_two = ""

In [13]:
for _, text, __ in img_two_text:
  final_text_two += " "
  final_text_two += text
final_text_two

' Healwell Dosage 25 gms 4 tablets 3-4 times a day or as directed by the physician. HOMOEOPATHIC MEDICINE KEEP IN A COOL & DRY PLACE Mfg. Lic. No. GH/25 Batch No. BC-829 Mfg. Date MRP Rs. AUG-20 (Incl. of all taxes)   90/- Date of Expiry 5 years from the date of manufacturing: BIO-COMBINATION Tablets'

In [14]:
medicine_name_match = re.search(r'(.*?) Dosage (\d+ gms \d+ tablets \d+-\d+ times a day)', final_text_two)
if medicine_name_match:
    medicine_name = medicine_name_match.group(1)
    dosage = medicine_name_match.group(2)
else:
    medicine_name = "Not found"
    dosage = "Not found"

manufacturing_date_match = re.search(r'Mfg\. Date MRP Rs\. (\w+-\d+)', final_text_two)
if manufacturing_date_match:
    manufacturing_date = manufacturing_date_match.group(1)
else:
    manufacturing_date = "Not found"

expiry_match = re.search(r'Date of Expiry (\d+ years from the date of manufacturing):', final_text_two)
if expiry_match:
    expiry_info = "Expiry " + expiry_match.group(1) + " years from the date of manufacturing"
else:
    expiry_info = "Not found"

ingredients_match = re.search(r'Ingredients image', final_text_two, re.IGNORECASE)
if ingredients_match:
    ingredients_image = "Mention of Ingredients Image Found"
else:
    ingredients_image = "Not mentioned"

In [17]:
print("Medicine Name:", medicine_name)
print("Dosage:", dosage)
print("Manufacturing Date:", manufacturing_date)
print("Expiry Date:", expiry_match)
print("Ingredients:", ingredients_match)

Medicine Name:  Healwell
Dosage: 25 gms 4 tablets 3-4 times a day
Manufacturing Date: AUG-20
Expiry Date: <re.Match object; span=(222, 276), match='Date of Expiry 5 years from the date of manufactu>
Ingredients: None
